# Actividad Integradora

**Modelación de sistemas multiagentes con gráficas computacionales** 
- Gamaliel Abisay Palomo Briones  
- Omar Mendoza Montoya  
- Guillermo Gabriel Rivas Aguilar 
   
24 de noviembre de 2022  

### Importar librerías

In [219]:
import agentpy as ap 
import numpy as np 
import matplotlib.pyplot as plt
import math
import IPython
import random

### Definir las clases de agentes

##### Clase de Semáforo

In [220]:
class TrafficLight(ap.Agent):
    def setup(self):
        self.curr_time = .1
        self.direction = [0, 1]

        self.color = 0 #0 -> verde, 1 -> amarillo, 2 -> rojo 
        self.color_time = 0

        self.green_time = 50
        self.yellow_time = 5
        self.red_time = 60

    def set_green(self):
        self.color = 0
        self.color_time = 0
       
    def set_yellow(self):
        self.color = 1
        self.color_time = 0

    def set_red(self):
        self.color = 2
        self.color_time = 0
    
    def update(self):
        self.color_time += self.curr_time

        if self.color == 0:
            if self.color_time >= self.green_time :
                self.set_yellow()
        
        if self.color == 1:
            if self.color_time >= self.yellow_time :
                self.set_red()

        if self.color == 2:
            if self.color_time >= self.red_time :
                self.set_green()


##### Clase de Auto

In [221]:
class Car(ap.Agent):
    
    def setup(self):
        self.curr_time = 0.1
        self.direction = [0, 1]
        self.speed = 0.0
        self.max_speed = 16.666
        self.state = 0 #0 -> regular, 1 -> chocado

    def update_position(self):
        
        if self.state == 1:
            return

        self.model.avenue.move_by(self, [self.speed * self.curr_time * self.direction[0], self.speed * self.curr_time * self.direction[1]])

    def update_speed_util(self, tl_separation, tl_state):
        if tl_separation < 2:
            self.speed = 0
            self.state = 0

        elif tl_separation < 10:
            self.speed = np.maximum(self.speed - 10 * self.curr_time, 0)
        
        elif tl_separation < 15:
            self.speed = np.maximum(self.speed - 5 * self.curr_time, 0)

        elif tl_separation < 20 and tl_state == 1:
            self.speed = np.minimum(self.speed - 2.5 * self.curr_time,self.max_speed)
            
        elif tl_separation < 30 and tl_state == 1:
            self.speed = np.maximum(self.speed - 2.5 * self.curr_time,5)

        elif tl_separation < 40 and tl_state == 1:
            self.speed = np.maximum(self.speed - 2.5 * self.curr_time,0)

        elif tl_separation < 50 and tl_state == 1:
            self.speed = np.maximum(self.speed - 2.5 * self.curr_time,5)

        else:
            self.speed = np.minimum(self.speed * 2.0 * self.curr_time, self.max_speed)

    def update_speed(self):
        if self.state == 1:
            return

        pos_init = self.model.avenue.positions[self]
        car_separation = 1000000000000
        
        for car in self.model.cars:
            if car == self:
                continue

            pos_final = self.model.avenue.positions[car]
            dot_p1 = self.direction[0] * car.direction[0] + self.direction[1] * car.direction[1]
            dot_p2 = (pos_final[0] - pos_init[0]) * self.direction[0] + (pos_final[1] - pos_init[1]) * self.direction[1] ##aqui no estoy segura

            if dot_p1 > 0 and dot_p2 > 0:
                d = math.sqrt((pos_final[0] - pos_init[0])**2 + (pos_final[1] - pos_init[1])**2)
                if d < car_separation:
                    car_separation = d

        traffic_light_state = 0
        traffic_light_separation = 1000000000000
        
        for traffic_light in self.model.traffic_lights:
            dot_p1 = self.direction[0] * traffic_light.direction[0] + self.direction[1] * traffic_light.direction[1]
            pos_final = self.model.avenue.positions[traffic_light]
            dot_p2 = (pos_final[0] - pos_init[0]) * self.direction[0] + (pos_final[1] - pos_init[1]) * self.direction[1] ##aqui no estoy segura

            if dot_p1 < 0 and dot_p2 > 0:
                d = math.sqrt((pos_final[0] - pos_init[0])**2 + (pos_final[1] - pos_init[1])**2)
                if d < traffic_light_separation:
                    traffic_light_separation = d
                    traffic_light_state = traffic_light.color

        self.update_speed_util(traffic_light_separation, traffic_light_state)


##### Clase del Peatón

In [222]:
class Person(ap.Agent):
    def setup(self):
        self.curr_time = 0.1
        self.direction = [0, 1]
        self.state = 0 #0 -> esperando, 1 -> cruzando, 2 -> ha cruzado
        self.click_stop = 0 #0 -> no, 1 -> si
        self.speed = 2.5
    
    def update_position(self):
        if self.state == 2:
            return
        
        self.model.avenue.move_by(self, [self.speed * self.curr_time * self.direction[0], self.speed * self.curr_time * self.direction[1]])
    
    def press_traffic_light(self):
        if self.click_stop == 1:
            return
        
        


### Definir la clase del Modelo

In [223]:
class AvenueModel(ap.Model):
    
    def setup(self):

        self.cars = ap.AgentList(self, self.p.cars, Car)
        self.cars.curr_time = self.p.curr_time
        self.cars.speed = self.p.v0

        c_north = int(self.p.cars / 2)
        c_south = int(self.p.cars / 2 - c_north)

        for k in range(c_north):
            self.cars[k].direction = [0,1]

        for k in range(c_south):
            self.cars[c_north + k].direction = [0, -1]


        self.traffic_lights = ap.AgentList(self, 2, TrafficLight)
        self.traffic_lights.curr_time = self.p.curr_time
        self.traffic_lights.green_duration = self.p.green
        self.traffic_lights.yellow_duration = self.p.yellow
        self.traffic_lights.red_duration = self.p.red
        self.traffic_lights[0].direction = [0, 1]
        self.traffic_lights[1].direction = [0, -1]

        ##############################################################################
        self.avenue = ap.Space(self, shape = [60, self.p.size], torus = True)

        self.avenue.add_agents(self.traffic_lights, random = True)
        self.avenue.move_to(self.traffic_lights[0], [10, self.p.size * 0.5 - 5 ])
        self.avenue.move_to(self.traffic_lights[1], [50, self.p.size * 0.5 + 5 ])

        ####################################################################################
        self.avenue.add_agents(self.cars, random = True)

        for k in range(c_north):
            self.avenue.move_to(self.cars[k], [40, 10 * (k + 1)])
        
        for k in range(c_south):
            self.avenue.move_to(self.cars[k + c_north], [20, self.p.size - 10 * (k + 1)])
            

    def step(self):
        self.traffic_lights[0].update()
        self.traffic_lights[1].update()

        self.cars.update_position()
        self.cars.update_speed()

    def update(self):
        crashes = len(self.cars.select(self.cars.state == 0))
        self.record('Colisiones', crashes)

    def end(self):
        crashes = len(self.cars.select(self.cars.state == 0))
        self.report('Colisiones', crashes)


### Definir los parámetros

In [224]:
parameters = {
    'steps' : 1000,
    'curr_time' : 0.1,
    'size' : 300,
    'green' : 20,
    'yellow' : 5,
    'red' : 10,
    'cars' : 30,
    'v0' : 0
    
}

### Definir la animación

In [225]:
def animation_plot(m, ax):
    ax.set_title(f"Avenida t={m.p.curr_time:.2f}")

    colors = ["green", "yellow", "red"]

    pos_s1 = m.avenue.positions[m.traffic_lights[0]]
    ax.scatter(pos_s1, pos_s1, s=20, c=colors[m.traffic_lights[0].color])

    pos_s2 = m.avenue.positions[m.traffic_lights[1]]
    ax.scatter(pos_s2 , pos_s2 - 10, s=20, c=colors[m.traffic_lights[1].color])

    ax.set_xlim(0, m.avenue.shape[0])
    ax.set_ylim(0, m.avenue.shape[1])

    for car in m.cars:
        pos_c = m.avenue.positions[car]
        ax.scatter(pos_s2, pos_c, s=20, c="black")

    ax.set_axis_off()
    ax.set_aspect('equal', 'box')

### Realizar simulación

In [226]:
model = AvenueModel(parameters)
results = model.run()

Completed: 1000 steps
Run time: 0:00:01.199279
Simulation finished


In [227]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=20))